In [12]:
## Standard

import os
import sys
sys.path.append('../RD-systems-and-test-benches')

import math
from datetime import datetime

## Specific

from tqdm.notebook import tqdm
# import copy
# from concurrent.futures import ProcessPoolExecutor
# import networkx as nx
# from itertools import product
# from itertools import chain

## Data handling 

import pandas as pd
import numpy as np
import openpyxl as opxl
from openpyxl.utils.dataframe import dataframe_to_rows

import sklearn.metrics
import scipy.constants as scc
import seaborn as sns
import scipy.integrate as integrate
import scipy.optimize as sco

## Plot

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import scienceplots
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('retina')
plt.style.use(['science'])

import plotly.graph_objects as go
import plotly.io as pio

from IPython.core.display import HTML

## Own modumes

import model as ty
import heat_transfer as bht
import proc as pr

import plot_functions_here as pfun
import general as gen

import thermal.SK_standard as nm
import utils.plot_functions as RD_pfun
import repo_config as rconfig

## Physics

import fluids as fds
import ht 

In [2]:
folder_path = r'G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\PVT-perf-1Dmodel'
geometry_path = os.path.join(folder_path, 'Panels\\Panel_V4.1I.xlsx')
hypotheses_path = folder_path + '\\' + 'Hypotheses\\Model_hypotheses.xlsx'
condi_path = folder_path + '\\' + 'Steady-state conditions inputs\\ssC_for_study.xlsx'
exp_path = folder_path + '\\' + 'Experiments\\Experiments-GMI.xlsx'

panelSpecs = pr.import_geometry(geometry_path)
hyp = pr.create_dict_from_excel(hypotheses_path,'Main')

steadyStateConditions_df = pd.read_excel(condi_path,header=0) ## Les deux première lignes ne sont pas à prendre en compte
steadyStateConditions_df.drop(index=[0], inplace=True)
steadyStateConditions_df.reset_index(drop=True, inplace=True)
# stepConditions = steadyStateConditions_df.to_dict(orient='records')

pr.adjust_steadyStateConditions(steadyStateConditions_df, hyp, convert_celsius=True)

In [ ]:
steadyStateConditions_df

In [4]:
panelSpecs['part2']['lambd_abs_ins'] = 0.
panelSpecs['part2']['lambd_tube_ins'] = 0.

pr.update_panelSpecs_after_modif(panelSpecs)

In [5]:
dic_names = {'T_in' : 'T_fluid_in',
             'T_out' : 'T_fluid_out',
             'Qdot_1_AG' : 'Qdot/AG',
             'T_amb' : 'T_amb',
             'u' : 'u'}

Normal non insulated

In [6]:
df_res, list_res = ty.simu_steadyStateConditions(panelSpecs, hyp, steadyStateConditions_df)

In [ ]:
df_res.columns

In [ ]:
ty.recap_residuals_steadyStateConditions(panelSpecs, df_res, list_res)

In [29]:
tab_mat,tab_res,X = nm.find_SK(df_res,dic_names,columns_list=[0,1,0,1,0,0,0,0,0],options=[],IAM=False,LSM='nnls')

In [ ]:
X

In [ ]:
X[0][1] - 3 * X[0][3]

In [ ]:
13.48-3*1.532

In [ ]:
ht_signs = {'Qdot_sun_PV':1, 'Qdot_sun_glass':1, 'Qdot_top_conv':-1, 'Qdot_top_rad':-1, 'Qdot_PV_sky':-1, 'Qdot_absfin_back_conv':-1, 'Qdot_absfin_back_rad':-1, 'Qdot_Base_back':-1, 'Qdot_tube_back_conv':-1, 'Qdot_tube_back_rad':-1, 'Qdot_tube_sky':-1}
ht_labels = ['Qdot_sun_PV', 'Qdot_sun_glass', 'Qdot_top_conv', 'Qdot_top_rad', 'Qdot_PV_sky', 'Qdot_absfin_back_conv', 'Qdot_absfin_back_rad', 'Qdot_Base_back', 'Qdot_tube_back_conv', 'Qdot_tube_back_rad', 'Qdot_tube_sky']
data = {key:ht_signs[key]*df_one[key].values[0] for key in ht_labels}

# Adjusted approach for a single-column histogram with offset bars
fig = go.Figure()

# Initialize cumulative sums for positive and negative values
cumulative_positive = 0
cumulative_negative = 0

for key, value in data.items():
    if value >= 0:
        # For positive values, start from the last positive cumulative sum
        base = cumulative_positive
        cumulative_positive += value
    else:
        # For negative values, start from the last negative cumulative sum
        base = cumulative_negative
        cumulative_negative += value
    
    fig.add_trace(go.Bar(
        x=["Heat Transfer Contributions"],
        y=[value],
        name=key,
        width=0.2,
        base=base  # Set the starting point of the bar
    ))

# Update the layout
fig.update_layout(
    title="Decomposition of Qdot_tube_fluid in Different Heat Transfer Contributions",
    yaxis_title="Value",
    barmode="overlay"  # Overlay the bars
)

# Show the figure
fig.show()


In [ ]:
df_res['u'].unique()

In [ ]:
df = df_res.copy()

df = df.loc[df['u'] == 0.8]

ht_signs = {'Qdot_sun_PV': 1, 'Qdot_sun_glass': 1, 'Qdot_top_conv': -1, 'Qdot_top_rad': -1, 'Qdot_PV_sky': -1, 'Qdot_absfin_back_conv': -1, 'Qdot_absfin_back_rad': -1, 'Qdot_Base_back': -1, 'Qdot_tube_back_conv': -1, 'Qdot_tube_back_rad': -1, 'Qdot_tube_sky': -1}
ht_labels = ['Qdot_sun_PV', 'Qdot_sun_glass', 'Qdot_top_conv', 'Qdot_top_rad', 'Qdot_PV_sky', 'Qdot_absfin_back_conv', 'Qdot_absfin_back_rad', 'Qdot_tube_back_conv', 'Qdot_tube_back_rad', 'Qdot_tube_sky']
colors_ht = [rconfig.get_hex_code('yellow',100), rconfig.get_hex_code('orange',100),rconfig.get_hex_code('green',150), rconfig.get_hex_code('blue',150),rconfig.get_hex_code('blue',100),rconfig.get_hex_code('green',100),rconfig.get_hex_code('grey',100),rconfig.get_hex_code('green',50),rconfig.get_hex_code('grey',50),rconfig.get_hex_code('blue',50)]

# Calculate the total positive heat transfers for each T_m - T_amb
df['positive_total'] = df[[label for label in ht_labels if ht_signs[label] == 1]].sum(axis=1)

# Initialize the figure
fig = go.Figure()

# Define the bar width
bar_width = 0.4

# Add traces for positive heat transfers (gains)
for i, label in enumerate(ht_labels):
    if ht_signs[label] == 1:
        fig.add_trace(go.Bar(
            x=df['T_m - T_amb'],
            y=df[label],
            name=label,
            width=[bar_width] * len(df),
            hovertext=round(df[label],0),
            hoverinfo='text',
            marker=dict(color=colors_ht[i])
        ))

# Add traces for negative heat transfers (losses) with base starting from the positive total
label_sum = []
for i, label in enumerate(ht_labels):
    if ht_signs[label] == -1:
        base = df['positive_total'] - df[[lbl for lbl in ht_labels if (ht_signs[lbl] == -1) & (lbl not in label_sum)]].sum(axis=1)
        fig.add_trace(go.Bar(
            x=df['T_m - T_amb'] + bar_width,  # Offset for side-by-side display
            y=df[label],
            name=label,
            width=[bar_width] * len(df),
            base=base,
            hovertext=round(df[label],0),
            hoverinfo='text',
            marker=dict(color=colors_ht[i])
        ))
        label_sum.append(label)


# fig.add_trace(go.Scatter(
#     x=df['T_m - T_amb'],
#     y=df['Qdot_exp'],
#     mode='lines',
#     name='Qdot exp',
#     line=dict(color='black', width=2)
# ))

# fig.add_trace(go.Scatter(
#     x=df['T_m - T_amb'],
#     y=df['Qdot_tube_fluid'],
#     mode='lines',
#     name='Qdot exp',
#     line=dict(color='grey', width=2)
# ))

# Update layout for stacked bars
fig.update_layout(barmode='relative', yaxis_title='Heat Transfer', xaxis_title='T_m - T_amb')


# Show the figure
fig.show()

In [ ]:
## Histogramme pour 1 cas

ht_signs = {'Qdot_sun_PV': 1, 'Qdot_sun_glass': 1, 'Qdot_top_conv': -1, 'Qdot_top_rad': -1, 'Qdot_PV_sky': -1, 'Qdot_absfin_back_conv': -1, 'Qdot_absfin_back_rad': -1, 'Qdot_Base_back': -1, 'Qdot_tube_back_conv': -1, 'Qdot_tube_back_rad': -1, 'Qdot_tube_sky': -1}
ht_labels = ['Qdot_sun_PV', 'Qdot_sun_glass', 'Qdot_top_conv', 'Qdot_top_rad', 'Qdot_PV_sky', 'Qdot_absfin_back_conv', 'Qdot_absfin_back_rad', 'Qdot_Base_back', 'Qdot_tube_back_conv', 'Qdot_tube_back_rad', 'Qdot_tube_sky']
data = {key: ht_signs[key] * df_one[key].values[0] for key in ht_labels}

# Création du graphique en barres
fig = go.Figure()

# Ajout de chaque barre sur le graphique
for key, value in data.items():
    fig.add_trace(go.Bar(x=[key], y=[value], name=key))

# Ajout des annotations sous chaque barre
for i, key in enumerate(data.keys()):
    fig.add_annotation(
        x=key,
        y=data[key],
        text="",
        showarrow=False,
    )

# Personnalisation du titre et des étiquettes
fig.update_layout(
    title="Decomposition of Qdot_tube_fluid in Different Heat Transfer Contributions",
    yaxis_title="Value",
    barmode="group"  # Pour afficher les barres côte à côte
)

Fins

In [9]:
panelSpecs['part2']['Heta'] = 20e-3
panelSpecs['part2']['N_ail'] = 100
panelSpecs['part2']['D'] = 20e-3

panelSpecs['part2']['fin_0'] = 1.
panelSpecs['part2']['N_f0'] = panelSpecs['part2']['N_ail']
panelSpecs['part2']['L_f0'] = panelSpecs['part2']['Heta'] 
panelSpecs['part2']['delta_f0'] = panelSpecs['part2']['w_tube'] + 2e-3
panelSpecs['part2']['delta_f0_int'] = panelSpecs['part2']['p_ext_tube'] / 2

pr.update_panelSpecs_after_modif(panelSpecs)

In [10]:
df_res, list_res = ty.simu_steadyStateConditions(panelSpecs, hyp, steadyStateConditions_df)

In [8]:
tab_mat,tab_res,X = nm.find_SK(df_res,dic_names,columns_list=[0,1,0,1,0,0,0,0,0],options=[],IAM=False,LSM='nnls')

In [ ]:
X

In [ ]:
19.3-1.63*3

In [ ]:
ty.recap_residuals_steadyStateConditions(panelSpecs, df_res, list_res)

In [ ]:
df_res.columns